## Multi-agent Collaboration

Create educational content using AG2

In [1]:
#%pip install --quiet ag2[gemini,openai]

In [1]:
GEMINI="gemini-2.0-flash"
OPENAI="gpt-4o-mini"
CLAUDE="claude-3-7-sonnet-latest"

import os
from dotenv import load_dotenv
load_dotenv("../keys.env")
assert os.environ["GEMINI_API_KEY"][:2] == "AI",\
       "Please specify the GEMINI_API_KEY access token in keys.env file"
#assert os.environ["ANTHROPIC_API_KEY"][:2] == "sk",\
#       "Please specify the ANTHROPIC_API_KEY access token in keys.env file"
assert os.environ["OPENAI_API_KEY"][:2] == "sk",\
       "Please specify the OPENAI_API_KEY access token in keys.env file"

## Set up content writers on different topics

Given a question, choose the content writer and have them create content on the topic. 

In [2]:
from autogen import ConversableAgent, LLMConfig
import os

llm_config = LLMConfig(
    api_type="google", # openai
    model=GEMINI, # OPENAI
    api_key=os.environ.get("GEMINI_API_KEY"), # OPENAI_API_KEY
    temperature=0.2,
)

In [3]:
# create agents to write history and math content
history_writer_prompt="""
You are a historian who is familiar with K-12 curriculum standards and writing styles.
Write 2 paragraphs to educate 9th grade students on the topic in a balanced way.
If feedback is provided, try to accomodate the feedback to some extent, but don't lose
sight of your audience and the goal of educating them with a balanced perspective.
"""
math_writer_prompt="""
You are a math teacher who is familiar with K-12 curriculum standards.
Provide clear explanations to show 9th grade students how to solve the given problem.
If feedback is provided, try to accomodate the feedback to some extent, but don't lose
sight of your audience and the goal of educating them in standard methods.
"""
task_assigner_prompt="""
You have two writers, one who is a historian and the other who is a math writer.
Given a question, assign it to the person who is likely to be better informed and able
to create content to answer it.
"""
with llm_config:
    history_writer = ConversableAgent(name="history_writer",
                                      system_message=history_writer_prompt)
    math_writer = ConversableAgent(name="math_writer",
                                      system_message=math_writer_prompt)
    human = ConversableAgent(name="human", human_input_mode="ALWAYS")
 

from pydantic import BaseModel
from typing import Literal
class TaskAssignmentResponse(BaseModel):
    writer: Literal['HISTORIAN', 'MATH WRITER']
    
llm_task_config = LLMConfig(
    api_type="google", # openai
    model=GEMINI, # OPENAI
    api_key=os.environ.get("GEMINI_API_KEY"), # OPENAI_API_KEY
    temperature=0.0,
    response_format=TaskAssignmentResponse
)
with llm_task_config:
    task_assigner = ConversableAgent(name="task_assigner",
                                      system_message=task_assigner_prompt)

In [4]:
def write_content_on_topic(question: str) -> str:
    # send topic to task assigner
    task_response = human.run(recipient=task_assigner,
                              message=question,
                              max_turns=1)
    return task_response

response = write_content_on_topic("Why was the Battle of Plassey so pivotal?")

In [5]:
response.messages

[{'content': 'Why was the Battle of Plassey so pivotal?',
  'role': 'assistant',
  'name': 'human'},
 {'content': '{\n  "writer": "HISTORIAN"\n}',
  'role': 'user',
  'name': 'task_assigner'}]

In [6]:
import json
json.loads(response.messages[-1]['content'])['writer']

'HISTORIAN'

In [7]:
import json
def write_content_on_topic(question: str) -> str:
    # send topic to task assigner
    task_response = human.run(recipient=task_assigner,
                              message=question,
                              max_turns=1)
    task_response.process()
    print(task_response.messages)
    writer = json.loads(task_response.messages[-1]['content'])['writer']
    if writer == 'HISTORIAN':
        writer = history_writer
    else:
        writer = math_writer
        
    # first version of content
    content_response = task_assigner.run(recipient=writer,
                                         message=question,
                                         max_turns=1)
    content_response.process()
    return question, content_response.messages[-1]['content']

question, answer = write_content_on_topic("Why was the Battle of Plassey so pivotal?")

human (to task_assigner):

Why was the Battle of Plassey so pivotal?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
task_assigner (to human):

{
  "writer": "HISTORIAN"
}

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (f6ec1e5f-7279-4550-ba13-902c99e01e38): Maximum turns (1) reached
[{'content': 'Why was the Battle of Plassey so pivotal?', 'role': 'assistant', 'name': 'human'}, {'content': '{\n  "writer": "HISTORIAN"\n}', 'role': 'user', 'name': 'task_assigner'}]
task_assigner (to history_writer):

Why was the Battle of Plassey so pivotal?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
history_writer (to task_assigner):

Alright class, let's talk about the Battle of Plassey. This 1757 battle, fought in Bengal, India, wasn't just another skirmish; it's considered a major turning point in the history

In [8]:
print(answer)

Alright class, let's talk about the Battle of Plassey. This 1757 battle, fought in Bengal, India, wasn't just another skirmish; it's considered a major turning point in the history of British influence in India. On one side, you had the British East India Company, a trading company with its own private army, led by Robert Clive. On the other, was Siraj-ud-Daulah, the Nawab of Bengal, who was suspicious of the Company's growing power. The Company, hungry for more control over trade and resources, clashed with the Nawab, and through a combination of military strength, and, crucially, a lot of bribery and backroom deals with disaffected members of the Nawab's court, the British East India Company won a decisive victory.

So, why was Plassey so pivotal? It wasn't necessarily the scale of the battle itself; the actual fighting was relatively short. The real significance lies in what happened *after*. The victory allowed the British East India Company to install a puppet ruler, effectively c

In [9]:
question2, answer2 = write_content_on_topic("x^2+50=150. Solve for x")

human (to task_assigner):

x^2+50=150. Solve for x

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
task_assigner (to human):

{
  "writer": "MATH WRITER"
}

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (b8ab6595-fb49-4af9-a295-2250c1cdd67c): Maximum turns (1) reached
[{'content': 'x^2+50=150. Solve for x', 'role': 'assistant', 'name': 'human'}, {'content': '{\n  "writer": "MATH WRITER"\n}', 'role': 'user', 'name': 'task_assigner'}]
task_assigner (to math_writer):

x^2+50=150. Solve for x

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
math_writer (to task_assigner):

Alright class, let's solve this equation together. Our goal is to isolate 'x', which means getting 'x' by itself on one side of the equation.

**1. Understand the Equation**

We have the equation:
x² + 50 = 150

This reads as "x squared

## Set up a review panel to review content

These are not content experts. But they will have strong opinions on all kinds of things!

In [28]:
from autogen.agentchat.group.patterns import RoundRobinPattern, AutoPattern
from autogen.agentchat import initiate_group_chat
import random

reviewers = []
with llm_config:
    reviewers.append(ConversableAgent(name="bureaucrat_1",
                                      system_message="""
You are a school district representative who wants to ensure that materials are concise
and easy to understand. You want to keep the cost of printing and remedial education down."""))
    reviewers.append(ConversableAgent(name="bureaucrat_3",
                                      system_message="""
You are a stickler for formal language in all school content."""))
    reviewers.append(ConversableAgent(name="conservative_parent",
                                      system_message="""
You are a conservative parent who believes that history teaching should
emphasize patriotic narratives and civic virtue.
You want a focus on Western civilization and a more positive
view of American/European history, and want to downplay aspects like the history of
slavery, colonialism, or the impact of independence and social justice movements.
You want math content to focus on foundational skills with a focus on accuracy and efficiency in
calculations. You prioritize traditional algorithms to ensure
that students master fundamental mathematical concepts."""))
    reviewers.append(ConversableAgent(name="liberal_parent",
                                      system_message="""
You are a liberal parent who believes that history teaching should
emphasize individual agency, critical thinking, and the importance of diverse
perspectives in understanding the past. You also value the connection between
history and contemporary issues, highlighting themes of power, liberty, and
individual rights. You want math content to emphasize understanding of
mathematical concepts in real-world contexts, rather than procedural skill.
You want to encourage critical thinking and problem-solving."""))
    reviewers.append(ConversableAgent(name="bureaucrat_2",
                                      system_message="""
You are a school administrator who wants to ensure that materials are easy
to teach and that students will learn it effectively."""))
    reviewers.append(ConversableAgent(name="secretary",
                                      system_message="""
You are the secretary of a curriculum review panel. You will
summarize all the feedback provided by all the other reviewers on the panel
into specific directions to the writer of the articler so that they can
rewrite the article based on feedback.
"""))

pattern = RoundRobinPattern(
    initial_agent=reviewers[0],
    agents=reviewers,
    user_agent=None,
    group_manager_args={
        "llm_config": llm_config,  # for group manager
    }
)

reviews, context, last_agent = initiate_group_chat(
    pattern=pattern,
    max_rounds=len(reviewers)+1,
    messages=f"""
    You are part of a review panel that is reviewing the following
    content that will appear in a 9th grade workbook.
    Be polite, and accomodate other viewpoints. You can not walk out
    or issue ultimatums.
    The summary of everyone's feedback will be given to the writer
    and they will rewrite based on this feedback.
    
    **Question**
    {question}
    
    **Worked Answer**
    {answer}
    """
)

_User (to chat_manager):


    You are part of a review panel that is reviewing the following
    content that will appear in a 9th grade workbook.
    Be polite, and accomodate other viewpoints. You can not walk out
    or issue ultimatums.
    The summary of everyone's feedback will be given to the writer
    and they will rewrite based on this feedback.
    
    **Question**
    Why was the Battle of Plassey so pivotal?
    
    **Worked Answer**
    Alright class, let's talk about the Battle of Plassey. This 1757 battle, fought in Bengal, India, wasn't just another skirmish; it's considered a major turning point in the history of British influence in India. On one side, you had the British East India Company, a trading company with its own private army, led by Robert Clive. On the other, was Siraj-ud-Daulah, the Nawab of Bengal, who was suspicious of the Company's growing power. The Company, hungry for more control over trade and resources, clashed with the Nawab, and through a com

In [29]:
reviews.chat_history[-1]

{'content': 'Okay, based on the feedback provided, here\'s a summary of directions for the writer to revise the history article:\n\n**Overarching Goal:** Revise the article to ensure it promotes inclusive history education that fosters critical thinking, social justice, and an understanding of diverse perspectives.\n\n**Specific Revisions Needed:**\n\n1.  **Address "Whose Heritage?":**\n    *   Explicitly demonstrate how the curriculum represents the diverse experiences and contributions of *all* students and their ancestors.\n    *   Include content that acknowledges the painful realities and injustices faced by marginalized communities.\n    *   Ensure the curriculum reflects the full spectrum of human experience, both triumphs and tragedies.\n\n2.  **Promote Critical Thinking:**\n    *   Move beyond presenting a one-sided or sanitized version of history.\n    *   Incorporate activities and materials that encourage students to question assumptions, analyze evidence, and form their ow

## Rewrite content based on reviews

In [30]:
last_agent

In [37]:
# send the review to the original writer and have them rewrite
rewrite_response = last_agent.run(recipient=history_writer, max_turns=2,
              message=f"""
Please incorporate the feedback from a review panel to
update content you wrote for a 9th grade workbook.

You were asked to write the answer for this question:
{question}

You wrote the following:
{answer}

The reviewer panel has provided the following feedback:
{reviews.chat_history[-1]['content']}

Incorporate the feedback to rewrite the content.""")

In [38]:
rewrite_response.process()

secretary (to history_writer):


Please incorporate the feedback from a review panel to
update content you wrote for a 9th grade workbook.

You were asked to write the answer for this question:
Why was the Battle of Plassey so pivotal?

You wrote the following:
Alright class, let's talk about the Battle of Plassey. This 1757 battle, fought in Bengal, India, wasn't just another skirmish; it's considered a major turning point in the history of British influence in India. On one side, you had the British East India Company, a trading company with its own private army, led by Robert Clive. On the other, was Siraj-ud-Daulah, the Nawab of Bengal, who was suspicious of the Company's growing power. The Company, hungry for more control over trade and resources, clashed with the Nawab, and through a combination of military strength, and, crucially, a lot of bribery and backroom deals with disaffected members of the Nawab's court, the British East India Company won a decisive victory.

So, why wa

In [40]:
rewrite_response.messages[-1]['content']

'Okay class, let\'s delve into the Battle of Plassey. This 1757 battle in Bengal, India, is often presented as a pivotal moment in the rise of British power, but it\'s crucial to understand it within a broader context of power struggles and the devastating impact of colonialism on the Bengali people. On one side stood the British East India Company, a powerful trading entity with its own army, led by Robert Clive. On the other was Siraj-ud-Daulah, the Nawab of Bengal, who rightly viewed the Company\'s growing influence with suspicion. The Company, driven by a desire for greater control over Bengal\'s lucrative trade, clashed with the Nawab. While military strength played a role, the Company\'s victory was significantly aided by bribery and secret deals with disaffected members of the Nawab\'s court, highlighting the complex web of alliances and betrayals that characterized this period. This betrayal, particularly that of Mir Jafar, resonates even today in Bengali culture, sometimes use

## Put it all together

In [3]:
from autogen import ConversableAgent, LLMConfig
from autogen.agentchat.group.patterns import RoundRobinPattern, AutoPattern
from autogen.agentchat import initiate_group_chat
import os, random, json
from pydantic import BaseModel
from typing import Literal

class TaskAssignmentResponse(BaseModel):
    writer: Literal['HISTORIAN', 'MATH WRITER']

def write_content_on_topic(question: str) -> str:
    # set up writers
    llm_config = LLMConfig(
        api_type="google", # openai
        model=GEMINI, # OPENAI
        api_key=os.environ.get("GEMINI_API_KEY"), # OPENAI_API_KEY
        temperature=0.2,
    )
    
    # create agents to write history and math content
    history_writer_prompt="""
    You are a historian who is familiar with K-12 curriculum standards and writing styles.
    Write 2 paragraphs to educate 9th grade students on the topic in a balanced way.
    If feedback is provided, try to accomodate the feedback to some extent, but don't lose
    sight of your audience and the goal of educating them with a balanced perspective.
    """
    math_writer_prompt="""
    You are a math teacher who is familiar with K-12 curriculum standards.
    Provide clear explanations to show 9th grade students how to solve the given problem.
    If feedback is provided, try to accomodate the feedback to some extent, but don't lose
    sight of your audience and the goal of educating them in standard methods.
    """
    task_assigner_prompt="""
    You have two writers, one who is a historian and the other who is a math writer.
    Given a question, assign it to the person who is likely to be better informed and able
    to create content to answer it.
    """
    with llm_config:
        history_writer = ConversableAgent(name="history_writer",
                                          system_message=history_writer_prompt)
        math_writer = ConversableAgent(name="math_writer",
                                          system_message=math_writer_prompt)
        human = ConversableAgent(name="human", human_input_mode="ALWAYS")
 


    # identify topic and choose the writer
    llm_task_config = LLMConfig(
        api_type="google", # openai
        model=GEMINI, # OPENAI
        api_key=os.environ.get("GEMINI_API_KEY"), # OPENAI_API_KEY
        temperature=0.0,
        response_format=TaskAssignmentResponse
    )
    with llm_task_config:
        task_assigner = ConversableAgent(name="task_assigner",
                                          system_message=task_assigner_prompt)
    task_response = human.run(recipient=task_assigner,
                              message=question,
                              max_turns=1)
    task_response.process()
    print(task_response.messages)
    writer = json.loads(task_response.messages[-1]['content'])['writer']
    if writer == 'HISTORIAN':
        writer = history_writer
    else:
        writer = math_writer
        
    # first version of content
    content_response = task_assigner.run(recipient=writer,
                                         message=question,
                                         max_turns=1)
    content_response.process()
    answer = content_response.messages[-1]['content']

    # set up review panel
    reviewers = []
    with llm_config:
        reviewers.append(ConversableAgent(name="bureaucrat_1",
                                          system_message="""
    You are a school district representative who wants to ensure that materials are concise
    and easy to understand. You want to keep the cost of printing and remedial education down."""))
        reviewers.append(ConversableAgent(name="bureaucrat_3",
                                          system_message="""
    You are a stickler for formal language in all school content."""))
        reviewers.append(ConversableAgent(name="conservative_parent",
                                          system_message="""
    You are a conservative parent who believes that history teaching should
    emphasize patriotic narratives and civic virtue.
    You want a focus on Western civilization and a more positive
    view of American/European history, and want to downplay aspects like the history of
    slavery, colonialism, or the impact of independence and social justice movements.
    You want math content to focus on foundational skills with a focus on accuracy and efficiency in
    calculations. You prioritize traditional algorithms to ensure
    that students master fundamental mathematical concepts."""))
        reviewers.append(ConversableAgent(name="liberal_parent",
                                          system_message="""
    You are a liberal parent who believes that history teaching should
    emphasize individual agency, critical thinking, and the importance of diverse
    perspectives in understanding the past. You also value the connection between
    history and contemporary issues, highlighting themes of power, liberty, and
    individual rights. You want math content to emphasize understanding of
    mathematical concepts in real-world contexts, rather than procedural skill.
    You want to encourage critical thinking and problem-solving."""))
        reviewers.append(ConversableAgent(name="bureaucrat_2",
                                          system_message="""
    You are a school administrator who wants to ensure that materials are easy
    to teach and that students will learn it effectively."""))
        reviewers.append(ConversableAgent(name="secretary",
                                          system_message="""
    You are the secretary of a curriculum review panel. You will
    summarize all the feedback provided by all the other reviewers on the panel
    into specific directions to the writer of the articler so that they can
    rewrite the article based on feedback.
    """))

    pattern = RoundRobinPattern(
        initial_agent=reviewers[0],
        agents=reviewers,
        user_agent=None,
        group_manager_args={
            "llm_config": llm_config,  # for group manager
        }
    )

    reviews, context, last_agent = initiate_group_chat(
        pattern=pattern,
        max_rounds=len(reviewers)+1,
        messages=f"""
        You are part of a review panel that is reviewing the following
        content that will appear in a 9th grade workbook.
        Be polite, and accomodate other viewpoints. You can not walk out
        or issue ultimatums.
        The summary of everyone's feedback will be given to the writer
        and they will rewrite based on this feedback.

        **Question**
        {question}

        **Worked Answer**
        {answer}
        """
    )
    
    # Rewrite article based on feedback from review panel
    rewrite_response = last_agent.run(recipient=writer, max_turns=2,
                  message=f"""
    Please incorporate the feedback from a review panel to
    update content you wrote for a 9th grade workbook.

    You were asked to write the answer for this question:
    {question}

    You wrote the following:
    {answer}

    The reviewer panel has provided the following feedback:
    {reviews.chat_history[-1]['content']}

    Incorporate the feedback to rewrite the content.""")
    rewrite_response.process()
    return rewrite_response.messages[-1]['content']

write_content_on_topic("Why was the Battle of Plassey so pivotal?")

human (to task_assigner):

Why was the Battle of Plassey so pivotal?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
task_assigner (to human):

{
  "writer": "HISTORIAN"
}

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (b964b8e3-ad0d-411b-b814-57bc8b4226c6): Maximum turns (1) reached
[{'content': 'Why was the Battle of Plassey so pivotal?', 'role': 'assistant', 'name': 'human'}, {'content': '{\n  "writer": "HISTORIAN"\n}', 'role': 'user', 'name': 'task_assigner'}]
task_assigner (to history_writer):

Why was the Battle of Plassey so pivotal?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
history_writer (to task_assigner):

The Battle of Plassey, fought in 1757, is often seen as a major turning point in the history of British influence in India. On one side was the British East India Company, led by R

"The Battle of Plassey in 1757 marked a turning point in the history of British influence in India. The British East India Company, led by Robert Clive, faced Siraj-ud-Daulah, the Nawab of Bengal. The Company's victory was aided by internal divisions and disagreements within the Nawab's court, which the Company was able to use to its advantage. This victory allowed the British East India Company to consolidate its power and expand its control over Indian territories, enabling it to more effectively manage its trade operations and generate revenue. The Company used the increased revenue to finance its growing military presence and administrative functions.\n\nWhile the British presence led to some advancements in infrastructure, such as the development of railways and irrigation systems in certain regions, it's important to acknowledge that this also involved the significant use of Indian resources and labor, contributing to long-term economic and social challenges for many Indians. Bec

In [4]:
write_content_on_topic("Solve for x in the equation x^2 + 50 = 150")

human (to task_assigner):

Solve for x in the equation x^2 + 50 = 150

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
task_assigner (to human):

{
  "writer": "MATH WRITER"
}

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (278cc9d4-cb50-43d7-8b39-2c92c7ca36d5): Maximum turns (1) reached
[{'content': 'Solve for x in the equation x^2 + 50 = 150', 'role': 'assistant', 'name': 'human'}, {'content': '{\n  "writer": "MATH WRITER"\n}', 'role': 'user', 'name': 'task_assigner'}]
task_assigner (to math_writer):

Solve for x in the equation x^2 + 50 = 150

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
math_writer (to task_assigner):

Alright class, let's solve this equation together. Our goal is to isolate 'x' and figure out what value(s) of 'x' make the equation true.

**Equation:**
x² + 50 = 150

**Step 1: I

'You\'re absolutely right! Thank you for catching my mistake in the "Your Turn" answer. I appreciate the feedback on the "Common Mistakes" section as well. Here\'s the final, polished version incorporating all the suggestions:\n\nOkay, class, let\'s tackle this equation together. Remember, in math, an **equation** is like a balanced scale. Both sides *must* always be equal! Our goal is to find the value(s) of \'x\' that keep the scale balanced. We call \'x\' a **variable**, and finding its value is called "solving" the equation. To do this, we want to **isolate the variable** \'x\' on one side of the equation.\n\n**Equation:**\n\nx² + 50 = 150\n\n**Step 1: Isolate the x² term**\n\nRight now, the x² term isn\'t alone. It has a "+ 50" hanging out with it. To get x² by itself, we need to "undo" the "+ 50". What\'s the opposite (or **inverse operation**) of adding 50? It\'s subtracting 50!\n\nBut here\'s the crucial part: to keep our equation balanced (like our scale!), whatever we do to o